In [44]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process, fuzz

tuition_df = pd.read_csv("college_tuition_data.csv")
earnings_df = pd.read_csv("college_earnings_data.csv")

tuition_df["Match Name"] = tuition_df["School Name"].str.lower().str.strip()
earnings_df["Match Name"] = earnings_df["School Name"].str.lower().str.strip()

matches = []
for name in tuition_df["Match Name"]:
    result = process.extractOne(name, earnings_df["Match Name"], scorer=fuzz.token_sort_ratio)
    if result is not None and len(result) >= 2:
        match = result[0]
        score = result[1]
        if score >= 97:
            matches.append({"tuition_name": name, "earnings_name": match})

matches_df = pd.DataFrame(matches)

tuition_matched = pd.merge(tuition_df, matches_df, left_on="Match Name", right_on="tuition_name")
merged_df = pd.merge(
    tuition_matched,
    earnings_df,
    left_on="earnings_name",
    right_on="Match Name",
    suffixes=("_tuition", "_earnings")
)

final_df = merged_df[[
    "School Name_tuition", "State",
    "In-State Tuition_tuition", "Out-of-State Tuition_tuition",
    "Median Earnings (10 yrs)", "Acceptance Rate",
    "SAT/ACT Scores", "Student Population", "Graduation Rate"
]]

final_df = final_df.rename(columns={
    "School Name_tuition": "School Name",
    "In-State Tuition_tuition": "In-State Tuition",
    "Out-of-State Tuition_tuition": "Out-of-State Tuition"
})

final_df["In-State Tuition"] = final_df["In-State Tuition"].replace({r'\$': '', r',': ''}, regex=True).astype(float)
final_df["Out-of-State Tuition"] = final_df["Out-of-State Tuition"].replace({r'\$': '', r',': ''}, regex=True).astype(float)
final_df["Student Population"] = final_df["Student Population"].replace({r',': ''}, regex=True).astype(float)
final_df["Graduation Rate"] = final_df["Graduation Rate"].str.replace('%', '').replace('-', np.nan).astype(float)
final_df["Acceptance Rate"] = final_df["Acceptance Rate"].str.replace('%', '').replace('-', np.nan).astype(float)

final_df["SAT Score"] = final_df["SAT/ACT Scores"].str.extract(r"(\d+)\sSAT")[0].astype(float)
final_df["ACT Score"] = final_df["SAT/ACT Scores"].str.extract(r"(\d+)\sACT")[0].astype(float)

final_df = final_df.drop(columns=["SAT/ACT Scores"])

final_df.to_csv("merged_college_data.csv", index=False)
print("Merged dataset saved to merged_college_data.csv")


Merged dataset saved to merged_college_data.csv
